# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as sc
from pprint import pprint
import hvplot.pandas
import seaborn as sns

# Import API keys
from api_keys import geoapify_key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5649,-52.7093,6.27,96,100,11.83,CA,1732915055
1,1,sisimiut,66.9395,-53.6735,-8.00,87,100,3.01,GL,1732915120
2,2,olonkinbyen,70.9221,-8.7187,-7.43,59,21,7.60,SJ,1732915121
3,3,lebu,-37.6167,-73.6500,15.07,71,2,6.43,CL,1732915122
4,4,waitangi,-43.9535,-176.5597,12.68,68,60,4.92,NZ,1732915123


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     579 non-null    int64  
 1   City        579 non-null    object 
 2   Lat         579 non-null    float64
 3   Lng         579 non-null    float64
 4   Max Temp    579 non-null    float64
 5   Humidity    579 non-null    int64  
 6   Cloudiness  579 non-null    int64  
 7   Wind Speed  579 non-null    float64
 8   Country     575 non-null    object 
 9   Date        579 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 45.4+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
 %%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
    )

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
mask = ((city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.4) & 
        (city_data_df.Cloudiness == 0))
ideal_city_df = city_data_df.loc[mask].reset_index(drop=True)
# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,69,lompoc,34.6391,-120.4579,21.19,58,0,2.57,US,1732915196
1,192,tazacorte,28.6290,-17.9293,22.54,37,0,3.13,ES,1732915335
2,338,progreso,21.2833,-89.6667,26.67,63,0,3.56,MX,1732915494
3,368,al jumum,21.6169,39.6981,23.65,63,0,3.33,SA,1732915529
4,398,alpine,32.8350,-116.7664,21.63,27,0,1.34,US,1732915560
5,402,tenenkou,14.4572,-4.9169,22.90,16,0,4.24,ML,1732915564
6,419,tshabong,-26.0500,22.4500,24.54,11,0,3.37,BW,1732915584
7,488,duba,27.3513,35.6901,21.75,33,0,2.21,SA,1732915659
8,495,santa ana,33.7456,-117.8678,23.95,24,0,2.06,US,1732915115
9,496,kamakwie,9.5000,-12.2333,24.10,37,0,0.86,SL,1732915518


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() # thanks Xpert AI

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = "None"

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lompoc,US,34.6391,-120.4579,58,None
1,tazacorte,ES,28.6290,-17.9293,37,None
2,progreso,MX,21.2833,-89.6667,63,None
3,al jumum,SA,21.6169,39.6981,63,None
4,alpine,US,32.8350,-116.7664,27,None
5,tenenkou,ML,14.4572,-4.9169,16,None
6,tshabong,BW,-26.0500,22.4500,11,None
7,duba,SA,27.3513,35.6901,33,None
8,santa ana,US,33.7456,-117.8678,24,None
9,kamakwie,SL,9.5000,-12.2333,37,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"
params = {
    "categories":categories,
    "limit":limit,
    #"filter":filters,
    #"bias":bias,
    "apiKey":geoapify_key 
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Get the city's name
    city = row["City"]
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: Red Roof Inn Lompoc
tazacorte - nearest hotel: App Leyma
progreso - nearest hotel: Centro Vacacional Obrero CTM
al jumum - nearest hotel: No hotel found
alpine - nearest hotel: Ayres Lodge Alpine
tenenkou - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
duba - nearest hotel: No hotel found
santa ana - nearest hotel: Holiday Inn Express
kamakwie - nearest hotel: No hotel found
mermoz boabab - nearest hotel: La maison blanche
dwarka - nearest hotel: The Dwarika Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
0,lompoc,US,34.6391,-120.4579,58,Red Roof Inn Lompoc
1,tazacorte,ES,28.6290,-17.9293,37,App Leyma
2,progreso,MX,21.2833,-89.6667,63,Centro Vacacional Obrero CTM
3,al jumum,SA,21.6169,39.6981,63,No hotel found
4,alpine,US,32.8350,-116.7664,27,Ayres Lodge Alpine
5,tenenkou,ML,14.4572,-4.9169,16,No hotel found
6,tshabong,BW,-26.0500,22.4500,11,No hotel found
7,duba,SA,27.3513,35.6901,33,No hotel found
8,santa ana,US,33.7456,-117.8678,24,Holiday Inn Express
9,kamakwie,SL,9.5000,-12.2333,37,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Hotel Name"]  # Add 'city' to the tooltip
    )

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)